In [42]:
import pandas as pd
from gensim import corpora, models
from gensim.corpora import Dictionary
from scipy.sparse import csr_matrix
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import keras as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import tensorflow as tf
import pymongo

In [43]:
client = pymongo.MongoClient(host="192.168.0.106", port=29197)
client["github"].authenticate("github", "git332", "github")
db = client["github"]

In [44]:
tag_set = [x["name"] for x in db.project_tag_more_than_100.find()]
len(tag_set)

116

In [45]:
lda = models.ldamodel.LdaModel.load('../model/readme_lda_256.model')

In [46]:
data_raw = pd.read_csv("../data/data_raw.csv", index_col=0)
data_raw.head(4)

,project_link,file_readme,file_npm,file_pypi,file_composer,JavaScript,HTML,CSS,Python,Shell,...,pytorch,cach,laravel,data-sci,natural-language-process,authent,computer-vis,compos,python3,yii2
0,/freeCodeCamp/freeCodeCamp,welcom freecodecamp org s open sourc codebas c...,1,0,0,0.941868,0.017145,0.039971,0.000000,0.001017,...,0,0,0,0,0,0,0,0,0,0
1,/vuejs/vue,support vue jsvue js mit licens open sourc pro...,1,0,0,0.976735,0.006227,0.003945,0.000000,0.001170,...,0,0,0,0,0,0,0,0,0,0
2,/twbs/bootstrap,bootstrap sleek intuit power end framework fas...,1,0,0,0.494735,0.182466,0.320497,0.000000,0.001512,...,0,0,0,0,0,0,0,0,0,0
3,/facebook/react,react middot react javascript librari build us...,1,0,0,0.951378,0.017036,0.003504,0.000079,0.001854,...,0,0,0,0,0,0,0,0,0,0


In [47]:
data_tag = np.array(data_raw[tag_set]).astype(np.int64)

In [48]:
data_raw.shape

(25467, 451)

In [49]:
readme_list = list(data_raw.file_readme)
corpus = []

for post in readme_list:
    
    corpus.append([word for word in str(post).strip().lower().split()])

len(readme_list)

25467

In [50]:
dictionary = Dictionary(corpus)
doc_bow = dictionary.doc2bow(corpus[0])
lda_list = lda[doc_bow]

In [51]:
corpus_lda = [lda[dictionary.doc2bow(doc)] for doc in corpus]

In [52]:
data_matrix = []

for post in corpus_lda:
    arr = np.zeros(256)
    for elem in post:
        arr[elem[0]] = elem[1]
    
    data_matrix.append(arr)

In [53]:
len(data_matrix)

25467

In [54]:
data_x_np = np.array(data_matrix)

In [55]:
len(data_x_np[1])

256

In [69]:
data_x_np.shape

(25467, 256)

In [56]:
len(tag_set)

116

In [57]:
import keras as K
from keras.layers.advanced_activations import PReLU, LeakyReLU, Softmax
init = K.initializers.glorot_uniform(seed=1)
simple_adam = K.optimizers.Adam(lr=0.0001)
model = K.models.Sequential()
model.add(K.layers.Dense(units=400, input_dim=256, kernel_initializer=init, activation=LeakyReLU(0.7)))
model.add(K.layers.Dense(units=600, kernel_initializer=init, activation=LeakyReLU(0.7)))
model.add(K.layers.Dense(units=400, kernel_initializer=init, activation=LeakyReLU(0.7)))
model.add(K.layers.Dense(units=116, kernel_initializer=init, activation=LeakyReLU(0.7)))
model.compile(loss='mean_squared_error', optimizer=simple_adam, metrics=['categorical_accuracy'])

/home/li/anaconda3/envs/ml/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [58]:
train_x, test_x, train_y, test_y = train_test_split(data_x_np, data_tag, train_size=0.75, test_size=0.25, random_state=1)

In [59]:
b_size = 512
max_epochs = 70
print("Starting training ")
h = model.fit(train_x, train_y, batch_size=b_size, epochs=max_epochs, shuffle=True, verbose=1)
print("Training finished \n")

Starting training 
Epoch 1/70
19100/19100 [==============================] - 1s 57us/step - loss: 0.0244 - categorical_accuracy: 0.1574
Epoch 2/70
19100/19100 [==============================] - 1s 43us/step - loss: 0.0198 - categorical_accuracy: 0.0824
Epoch 3/70
19100/19100 [==============================] - 1s 43us/step - loss: 0.0190 - categorical_accuracy: 0.0999
Epoch 4/70
19100/19100 [==============================] - 1s 43us/step - loss: 0.0185 - categorical_accuracy: 0.1074
Epoch 5/70
19100/19100 [==============================] - 1s 43us/step - loss: 0.0183 - categorical_accuracy: 0.1059
Epoch 6/70
19100/19100 [==============================] - 1s 43us/step - loss: 0.0181 - categorical_accuracy: 0.1053
Epoch 7/70
19100/19100 [==============================] - 1s 43us/step - loss: 0.0180 - categorical_accuracy: 0.1090
Epoch 8/70
19100/19100 [==============================] - 1s 44us/step - loss: 0.0179 - categorical_accuracy: 0.1105
Epoch 9/70
19100/19100 [=====================

In [60]:
p = model.predict(test_x)

In [61]:
p.shape

(6367, 116)

In [62]:
p[0], test_y[0]

(array([ 4.29666489e-02, -1.29527142e-02,  1.39985934e-01, -5.80065697e-03,
        -4.49155550e-03, -5.22969663e-03,  1.75532699e-03,  3.27765942e-04,
        -1.10133470e-03, -2.31501856e-03,  3.80135849e-02,  2.01412663e-03,
         8.44773114e-01,  8.45634460e-01,  8.43551993e-01,  8.43502402e-01,
         8.34188282e-01,  1.85817275e-02,  6.85833301e-03, -6.93724025e-03,
         2.60732509e-03,  1.41719794e-02, -4.19468945e-03, -6.24114042e-03,
         1.18450932e-02,  7.34416861e-03, -2.16477755e-02,  2.69990298e-03,
        -7.88816344e-03,  1.60345033e-01, -7.53240660e-03,  2.38639880e-02,
        -1.11724352e-02, -6.02110662e-03,  1.95212755e-02,  1.11076823e-02,
        -2.53582839e-04,  4.09822492e-03,  1.30392509e-02, -6.45760400e-03,
        -1.77486762e-02,  1.43989958e-02, -3.77497636e-05, -1.05583946e-04,
        -3.27275088e-03,  4.86800224e-02,  1.40287755e-02, -1.23394402e-02,
         2.08162796e-03,  1.22225366e-03, -2.64347135e-03,  6.40682923e-03,
        -5.6

In [63]:
# 取出分数不为0的项
p_list = []
for k in p:
    p_tag = {}
    for i, score in enumerate(k.tolist()):
        if(score!=0):
            p_tag[score] = i
    p_list.append(p_tag)

In [64]:
def recall(true_tag, pre_tag):
    t = 0
    for i in pre_tag:
        if(i in true_tag):
            t += 1
    return t/len(true_tag)

def precision(true_tag, pre_tag):
    t = 0
    for i in pre_tag:
        if(i in true_tag):
            t += 1
    return t/len(pre_tag)

In [65]:
p_list_top5 = []
for i in range(len(p_list)):
    p_list_top5.append([p_list[i][k] for k in sorted(p_list[i].keys())][-5:])
    
p_list_top10 = []
for i in range(len(p_list)):
    p_list_top10.append([p_list[i][k] for k in sorted(p_list[i].keys())][-10:])

In [66]:
true_tag_real = []
for row in test_y:
    true_tag_real.append(row.nonzero()[0])

In [67]:
recal_at_10_count = 0
recal_at_5_count = 0
precision_at_10_count = 0
precision_at_5_count = 0

for i in range(len(true_tag_real)):
    recal_at_10_count += recall(true_tag_real[i], p_list_top10[i])


for i in range(len(true_tag_real)):
    recal_at_5_count += recall(true_tag_real[i], p_list_top5[i])
    
for i in range(len(true_tag_real)):
    precision_at_10_count += precision(true_tag_real[i], p_list_top10[i])    

for i in range(len(true_tag_real)):
    precision_at_5_count += precision(true_tag_real[i], p_list_top5[i])
    
recall_at_10 = recal_at_10_count/len(true_tag_real)
recall_at_5 = recal_at_5_count/len(true_tag_real)
precision_at_10 = precision_at_10_count/len(true_tag_real)
precision_at_5 = precision_at_5_count/len(true_tag_real)

print("Recall@10: {}".format(recall_at_10))
print("Precision@10: {}".format(precision_at_10_count/len(true_tag_real)))
print("F1-score@10: {}".format((2*recall_at_10*precision_at_10)/(recall_at_10+precision_at_10)))
print("Recall@5: {}".format(recall_at_5))
print("Precision@5: {}".format(precision_at_5_count/len(true_tag_real)))
print("F1-score@5: {}".format((2*recall_at_5*precision_at_5)/(recall_at_5+precision_at_5)))

Recall@10: 0.7856114275581847
Precision@10: 0.2747918957122603
F1-score@10: 0.40716517712547895
Recall@5: 0.5301841549721252
Precision@5: 0.4450447620543411
F1-score@5: 0.4838980407061656


In [68]:
test_y[60]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])